# Car Plate Detection Model (InceptionResNetV2)

## Imports

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import cv2

from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import load_img, img_to_array

from tensorflow.keras.applications import MobileNetV2, InceptionV3, InceptionResNetV2
from tensorflow.keras.layers import Dense, Dropout, Flatten, Input
from tensorflow.keras.models import Model
import tensorflow as tf
from tensorflow.keras.callbacks import TensorBoard

## 1- Loading Data

In [2]:
df = pd.read_csv('labels.csv')
print(len(df))
df.head()

3000


,filepath,xmin,xmax,ymin,ymax
0,./dataset01/train\AN1.xml,94,185,364,385
1,./dataset01/train\AN10.xml,109,165,168,181
2,./dataset01/train\AN4.xml,82,141,229,245
3,./dataset01/train\AN5.xml,150,205,244,256
4,./dataset01/train\AN6.xml,121,184,229,246


In [3]:
img_path = df['filepath'].str.replace('.xml', '.png').to_list()
print(len(img_path))
img_path

3000


C:\Users\User\AppData\Local\Temp/ipykernel_11652/416220250.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  img_path = df['filepath'].str.replace('.xml', '.png').to_list()


['./dataset01/train\\AN1.png',
 './dataset01/train\\AN10.png',
 './dataset01/train\\AN4.png',
 './dataset01/train\\AN5.png',
 './dataset01/train\\AN6.png',
 './dataset01/train\\AN7.png',
 './dataset01/train\\AP1.png',
 './dataset01/train\\AP10.png',
 './dataset01/train\\AP12.png',
 './dataset01/train\\AP13.png',
 './dataset01/train\\AP14.png',
 './dataset01/train\\AP15.png',
 './dataset01/train\\AP16.png',
 './dataset01/train\\AP17.png',
 './dataset01/train\\AP18.png',
 './dataset01/train\\AP19.png',
 './dataset01/train\\AP2.png',
 './dataset01/train\\AP20.png',
 './dataset01/train\\AP21.png',
 './dataset01/train\\AP23.png',
 './dataset01/train\\AP24.png',
 './dataset01/train\\AP25.png',
 './dataset01/train\\AP26.png',
 './dataset01/train\\AP27.png',
 './dataset01/train\\AP29.png',
 './dataset01/train\\AP3.png',
 './dataset01/train\\AP30.png',
 './dataset01/train\\AP33.png',
 './dataset01/train\\AP34.png',
 './dataset01/train\\AP39.png',
 './dataset01/train\\AP4.png',
 './dataset01/tra

### verifing images and labels

In [4]:
img = cv2.imread(img_path[40])
print(img.shape)

cv2.namedWindow('example',cv2.WINDOW_NORMAL)
cv2.imshow('example',img)
cv2.waitKey(0)
cv2.destroyAllWindows()

(153, 272, 3)


In [5]:
img_path[40]

'./dataset01/train\\AR1.png'

In [6]:
print(df['xmin'][40],df['ymin'][40],df['xmax'][40],df['ymax'][40])

99 110 156 122


In [7]:
cv2.rectangle(img,(df['xmin'][40],df['ymin'][40]),(df['xmax'][40],df['ymax'][40]),(0,255,0),1)
cv2.namedWindow('example',cv2.WINDOW_NORMAL)
cv2.imshow('example',img)
cv2.waitKey(0)
cv2.destroyAllWindows()

## 2- Data Preprocessing

In [17]:
labels = df.iloc[:,1:].values
labels.shape

(3000, 4)

In [18]:
labels

array([[ 94, 185, 364, 385],
       [109, 165, 168, 181],
       [ 82, 141, 229, 245],
       ...,
       [108, 181, 247, 266],
       [ 98, 184, 210, 229],
       [ 92, 152, 204, 221]], dtype=int64)

In [19]:
norm_imgs = []
norm_labels = []

for ind in range(len(img_path)):
    
    image = img_path[ind]
    img_arr = cv2.imread(image)
    h,w,d = img_arr.shape
    h,w,d = float(h),float(w),float(d)
    
    # prepprocesing
    
    # uniform the size of images
    ## Inception_ResNetV2 requires input images with size 299x299 not 224x224, as per keras documentation.
    uni_size_image = cv2.resize(img_arr, (299,299))
    
    # normalization of both images and labels
    
    # images
    norm_load_image_arr = uni_size_image/255.0
    
    # labels
    xmin,xmax,ymin,ymax = labels[ind]
    nxmin,nxmax = xmin/w,xmax/w
    nymin,nymax = ymin/h,ymax/h
    label_norm = (nxmin,nxmax,nymin,nymax)
    
    # append
    norm_imgs.append(norm_load_image_arr)
    norm_labels.append(label_norm)

In [20]:
# test decoding

## TO_DOs in this testing
# > the resizing in preprocessing (size_uniformizatin), not resized back to the original images' sizes in testing.
# > denormalized_images.astype('uint8') ('uint8' to be readable using cv2.imread).
# > denormalization of labels through multiplication Xmin,max by w and Ymin,max by h.

decoded_img= (norm_imgs[40]*255).astype('uint8')

decoded_label = tuple([299*x for x in norm_labels[40]])
print(decoded_label)

cv2.rectangle(decoded_img,(int(decoded_label[0]),int(decoded_label[2])),(int(decoded_label[1]),int(decoded_label[3])),(0,255,0),1)
cv2.namedWindow('example',cv2.WINDOW_NORMAL)
cv2.imshow('example',decoded_img)
cv2.waitKey(0)
cv2.destroyAllWindows()

(108.82720588235294, 171.48529411764704, 214.96732026143792, 238.4183006535948)


## 3- Train-Val Split

In [21]:
X = np.array(norm_imgs,dtype=np.float32)
y = np.array(norm_labels,dtype=np.float32)

In [22]:
X.shape,y.shape

((3000, 299, 299, 3), (3000, 4))

In [23]:
x_train,x_test,y_train,y_test = train_test_split(X,y,train_size=0.8,random_state=42)
x_train.shape,x_test.shape,y_train.shape,y_test.shape

((2400, 299, 299, 3), (600, 299, 299, 3), (2400, 4), (600, 4))

## 4- Deep Learning Model

In [25]:
# InceptionResNetV2 Model

inception_resnet = InceptionResNetV2(weights="imagenet",include_top=False,
                                     input_tensor=Input(shape=(299,299,3)))
inception_resnet.trainable=False
# ---------------------
headmodel = inception_resnet.output
headmodel = Flatten()(headmodel)
headmodel = Dense(128,activation="relu")(headmodel)
headmodel = Dense(128,activation="relu")(headmodel)
headmodel = Dense(64,activation="relu")(headmodel)
headmodel = Dense(4,activation='sigmoid')(headmodel)
# ---------- model
model = Model(inputs=inception_resnet.input,outputs=headmodel)

219055592/219055592 [==============================] - 127s 1us/step


In [30]:
# complie model

model.compile(loss='mse',optimizer=tf.keras.optimizers.Adam(learning_rate=0.01))
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 299, 299, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 149, 149, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 149, 149, 32  96         ['conv2d[0][0]']                 
 alization)                     )                                                             

 batch_normalization_5 (BatchNo  (None, 35, 35, 96)  288         ['conv2d_5[0][0]']               
 rmalization)                                                                                     
                                                                                                  
 batch_normalization_7 (BatchNo  (None, 35, 35, 64)  192         ['conv2d_7[0][0]']               
 rmalization)                                                                                     
                                                                                                  
 batch_normalization_10 (BatchN  (None, 35, 35, 96)  288         ['conv2d_10[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 batch_normalization_11 (BatchN  (None, 35, 35, 64)  192         ['conv2d_11[0][0]']              
 ormalizat

 activation_21 (Activation)     (None, 35, 35, 32)   0           ['batch_normalization_21[0][0]'] 
                                                                                                  
 conv2d_19 (Conv2D)             (None, 35, 35, 32)   10240       ['block35_1_ac[0][0]']           
                                                                                                  
 conv2d_22 (Conv2D)             (None, 35, 35, 48)   13824       ['activation_21[0][0]']          
                                                                                                  
 batch_normalization_19 (BatchN  (None, 35, 35, 32)  96          ['conv2d_19[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 batch_normalization_22 (BatchN  (None, 35, 35, 48)  144         ['conv2d_22[0][0]']              
 ormalizat

                                                                                                  
 activation_24 (Activation)     (None, 35, 35, 32)   0           ['batch_normalization_24[0][0]'] 
                                                                                                  
 activation_26 (Activation)     (None, 35, 35, 32)   0           ['batch_normalization_26[0][0]'] 
                                                                                                  
 activation_29 (Activation)     (None, 35, 35, 64)   0           ['batch_normalization_29[0][0]'] 
                                                                                                  
 block35_3_mixed (Concatenate)  (None, 35, 35, 128)  0           ['activation_24[0][0]',          
                                                                  'activation_26[0][0]',          
                                                                  'activation_29[0][0]']          
          

 ormalization)                                                                                    
                                                                                                  
 batch_normalization_40 (BatchN  (None, 35, 35, 48)  144         ['conv2d_40[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 activation_37 (Activation)     (None, 35, 35, 32)   0           ['batch_normalization_37[0][0]'] 
                                                                                                  
 activation_40 (Activation)     (None, 35, 35, 48)   0           ['batch_normalization_40[0][0]'] 
                                                                                                  
 conv2d_36 (Conv2D)             (None, 35, 35, 32)   10240       ['block35_4_ac[0][0]']           
          

...

                                                                                                  
 activation_169 (Activation)    (None, 8, 8, 224)    0           ['batch_normalization_169[0][0]']
                                                                                                  
 conv2d_167 (Conv2D)            (None, 8, 8, 192)    399360      ['block8_1_ac[0][0]']            
                                                                                                  
 conv2d_170 (Conv2D)            (None, 8, 8, 256)    172032      ['activation_169[0][0]']         
                                                                                                  
 batch_normalization_167 (Batch  (None, 8, 8, 192)   576         ['conv2d_167[0][0]']             
 Normalization)                                                                                   
                                                                                                  
 batch_nor

 batch_normalization_175 (Batch  (None, 8, 8, 192)   576         ['conv2d_175[0][0]']             
 Normalization)                                                                                   
                                                                                                  
 batch_normalization_178 (Batch  (None, 8, 8, 256)   768         ['conv2d_178[0][0]']             
 Normalization)                                                                                   
                                                                                                  
 activation_175 (Activation)    (None, 8, 8, 192)    0           ['batch_normalization_175[0][0]']
                                                                                                  
 activation_178 (Activation)    (None, 8, 8, 256)    0           ['batch_normalization_178[0][0]']
                                                                                                  
 block8_4_

                                                                                                  
 activation_186 (Activation)    (None, 8, 8, 256)    0           ['batch_normalization_186[0][0]']
                                                                                                  
 block8_6_mixed (Concatenate)   (None, 8, 8, 448)    0           ['activation_183[0][0]',         
                                                                  'activation_186[0][0]']         
                                                                                                  
 block8_6_conv (Conv2D)         (None, 8, 8, 2080)   933920      ['block8_6_mixed[0][0]']         
                                                                                                  
 block8_6 (Lambda)              (None, 8, 8, 2080)   0           ['block8_5_ac[0][0]',            
                                                                  'block8_6_conv[0][0]']          
          

                                                                                                  
 block8_8 (Lambda)              (None, 8, 8, 2080)   0           ['block8_7_ac[0][0]',            
                                                                  'block8_8_conv[0][0]']          
                                                                                                  
 block8_8_ac (Activation)       (None, 8, 8, 2080)   0           ['block8_8[0][0]']               
                                                                                                  
 conv2d_196 (Conv2D)            (None, 8, 8, 192)    399360      ['block8_8_ac[0][0]']            
                                                                                                  
 batch_normalization_196 (Batch  (None, 8, 8, 192)   576         ['conv2d_196[0][0]']             
 Normalization)                                                                                   
          

 )                                                                                                
                                                                                                  
 conv_7b_ac (Activation)        (None, 8, 8, 1536)   0           ['conv_7b_bn[0][0]']             
                                                                                                  
 flatten (Flatten)              (None, 98304)        0           ['conv_7b_ac[0][0]']             
                                                                                                  
 dense (Dense)                  (None, 128)          12583040    ['flatten[0][0]']                
                                                                                                  
 dense_1 (Dense)                (None, 128)          16512       ['dense[0][0]']                  
                                                                                                  
 dense_2 (

In [32]:
tfb = TensorBoard('object_detection')

In [35]:
history = model.fit(x=x_train,y=y_train,batch_size=32,epochs=2,
                    validation_data=(x_test,y_test),callbacks=[tfb])

Epoch 1/2
75/75 [==============================] - 574s 8s/step - loss: 0.1748 - val_loss: 0.1730
Epoch 2/2
75/75 [==============================] - 444s 6s/step - loss: 0.1748 - val_loss: 0.1730


In [36]:
model.save('./models/object_detection_v05.h5')

### end